# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  

`cv2.fillPoly()` for regions selection  

`cv2.line()` to draw lines on an image given endpoints  

`cv2.addWeighted()` to coadd / overlay two images

`cv2.cvtColor()` to grayscale or change color

`cv2.imwrite()` to output images to file  

`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

DEBUG = False  # Set it to True if you want to print DEBUG information.

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_region(img, vertices):
    """Draws the region of interest delimited by vertices on img"""
    masked_image = region_of_interest(img, vertices)
    plt.imshow(masked_image)
    

def get_slope(line):
    """Calculates the slope of a line"""
    x1,y1,x2,y2 = line
    return (y2 - y1) / (x2 - x1)

def build_line(slopes, xs, ys, y_min, y_max):
    """
    Builds a line that represents the average line of the input parameters.
    """

    if slopes != []:
        m = np.mean(slopes)
    else:
        m = 1

    if xs != []:
        x_mean = np.mean(xs)
    else:
        x_mean = 1

    if ys != []:
        y_mean = np.mean(ys)
    else:
        y_mean = y_min

    b = y_mean - m * x_mean
    x_max = (y_max - b) / m
    x_min = (y_min - b) / m
    
    if DEBUG:
        print("slopes:", slopes)
        print("m:", m)
        print("xs:", xs)
        print("x_mean:", x_mean)
        print("ys:", ys)
        print("y_mean:", y_mean)
        print("b:", b)
        print("x_max:",x_max)
        print("x_min:",x_min)
        
    return [x_min, y_min, x_max, y_max]
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    This function draws solid `lines` with `color` and `thickness`.
    """
    # Left lane accumulators.
    left_slopes = []
    left_xs = []
    left_ys = []
    
    # Right lane accumulators.
    right_slopes = []
    right_xs = []
    right_ys = []
    
    for l in lines:
        line = l[0]
        x1, y1, x2, y2 = line  # Unpack
        m = get_slope(line)
        
        # Only consider lines with slopes within this range. 
        if 0.2 <= abs(m) <= 0.8: 
            if m > 0:
                left_slopes.append(m)
                left_xs += [x1, x2]
                left_ys += [y1, y2]
            else:
                right_slopes.append(m)
                right_xs += [x1, x2]
                right_ys += [y1, y2]
            
    y_min_global = min(left_ys + right_ys)
    y_max_global = img.shape[0]

    
    # Build left line
    lx1, ly1, lx2, ly2 = build_line(left_slopes, left_xs, left_ys, y_min_global, y_max_global)

    # Build right line
    rx1, ry1, rx2, ry2 = build_line(right_slopes, right_xs, right_ys, y_min_global, y_max_global)

    
    if DEBUG:
        print("lx1:", lx1, "ly1:", ly1, "lx2:", lx2, "ly2:", ly2)
        print("rx1:", rx1, "ry1:", ry1, "rx2:", rx2, "ry2:", ry2)

    # Draw left and right lane lines.
    cv2.line(img, (math.floor(lx1), math.floor(ly1)), (math.floor(lx2), math.floor(ly2)), color, thickness)
    cv2.line(img, (math.ceil(rx1), math.ceil(ry1)), (math.ceil(rx2), math.ceil(ry2)), color, thickness)
    
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns a blank image to draw on and an array of lines found by Hough Transform algorithm.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)

    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

Here's our pipeline definition:



In [ ]:
def find_lane_lines(image, parameters=None):
    """
    Finds lane lines on a image of a road.
    
    The format of the parameters dict must be as follows (example):
    parameters = {
        # Canny Edge parameters.
        'low_threshold': 60,
        'high_threshold': 180,
        # Gaussian blur parameters.
        'kernel_size': 7,
        # Masked region parameters.
        'vertices': np.array([[(110, Y_length), (410, 350), (570, 350), (900, Y_length)]], dtype=np.int32),
        # Hough Transform parameters.
        'rho': 1,
        'theta': np.pi / 180,
        'hough_threshold': 10,
        'min_line_length': 15,
        'max_line_gap': 20
    }
    
    If parameters is None then defaults will be used. If only some paremeters are passed, they will be used along
    with defaults for those undefined values.
    """
    Y_length, X_length, _ = image.shape
    
    defaults = {
        'low_threshold': 50,
        'high_threshold': 150,
        'kernel_size': 3,
        'vertices':np.array([[(107, Y_length), (460, 311), (500, 311), (930, Y_length)]], dtype=np.int32),
        'rho': 1,
        'theta': np.pi / 180,
        'hough_threshold': 20,
        'min_line_length': 100,
        'max_line_gap': 160,
        'line_thickness': 3
    }
    
    if not parameters:
        # Just use the defaults.
        params = defaults
    else:
        # Merge input parameters and defaults.
        params = {}
        params['low_threshold'] = parameters.get('low_threshold', defaults['low_threshold'])
        params['high_threshold'] = parameters.get('high_threshold', defaults['high_threshold'])
        params['kernel_size'] = parameters.get('kernel_size', defaults['kernel_size'])
        params['vertices'] = parameters.get('vertices', defaults['vertices'])
        params['rho'] = parameters.get('rho', defaults['rho'])
        params['theta'] = parameters.get('theta', defaults['theta'])
        params['hough_threshold'] = parameters.get('hough_threshold', defaults['hough_threshold'])
        params['min_line_length'] = parameters.get('min_line_length', defaults['min_line_length'])
        params['max_line_gap'] = parameters.get('max_line_gap', defaults['max_line_gap'])
        params['line_thickness'] = parameters.get('line_thickness', defaults['line_thickness'])

    if DEBUG:
        print("PARAMETERS:", params)
        
    # Let's turn it into grayscale:
    gray_image = grayscale(image)
    
    # Now, let's blur it
    blurred_image = gaussian_blur(gray_image, params['kernel_size'])
    
    # Apply Canny Edges transform:
    edges_image = canny(blurred_image, params['low_threshold'], params['high_threshold'])
    
    masked_image = region_of_interest(edges_image, params['vertices'])
    
    # Get the image and the lanes lines that'll be drawn on top of it.
    lines_img, lines = hough_lines(masked_image, params['rho'], params['theta'], params['hough_threshold'], params['min_line_length'], params['max_line_gap'])
    
    draw_lines(lines_img, lines, thickness=params['line_thickness'])
    
    # Return the original image with lane lines drawed on top of it.
    return weighted_img(lines_img, image)
    

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
for img in os.listdir("test_images/"):
    image = mpimg.imread("test_images/" + img)
    Y_length, X_length, _ = image.shape    
    cv2.imwrite("test_images_copy/copy_" + img, find_lane_lines(image))

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image, parameters=None):
    """
    Takes an image of a road and draws lanes on top of it, which identify the lane lines.
    """
    if DEBUG:
        draw_region(image,parameters['vertices'])
    return find_lane_lines(image, parameters)

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


___
_Although the current pipeline works well for the test videos and images, there are many places where further improvement would be very welcomed, starting for the overall elegance of the code. Most functions do not perform necessary checks on the input data, which leads to runtime errors very difficult to spot. It would be great to use a method to keep the lines a little bit steadier. One possible technique to explore is to take the average of each line on every frame._

_There are occassions where some wild NaNs emerge and impede the correct processing of the image. The region mask isn't very wide, so a different camera setting or placing would probably break the pipeline._

_The pipeline doesn't handle very well shadows on the road. As an example of this situation, you can reproduce the annotated challenge video and in the interval 00:04-00:06 the lines are all over the place._

_I found this project very challenging but fun and rewarding as well. I imagine this is not how a real Self Driven Car spot the lines on the road, so I'm really curious about the nature of the actual method. Nevertheless, it has been a great exercise._
___

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(lambda img: process_image(img, parameters={'kernel_size': 5, 'vertices': np.array([[(300, 650), (610, 420), (700, 420), (1060, 650)]], dtype=np.int32)}))
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))